In [ ]:
import openai
import os
import pandas
import re

## To-Do list

* giving instructions to chatgpt, asking it to generate a message and choose an action (play the game)
* based on the message generated, we ask for a response (it could also be null I think)
* we need to save these messages and actions in a dataframe for every round
* given the chat to chatgpt, we give another instruction to generate the new message based on the previous messages and actions

  * if the chat can be kept going on without interfering, we would not need the second prompt text
  * if we present the other player's message and action, we would need some nlp codes.

In [ ]:
openai.api_key = "API KEY"

# Set up the GPT-3 model and parameters
model_engine = "davinci" # Change this to another model if desired
max_words = 100 # Maximum number of tokens per prompt
creativity = 0.75 # Controls the "creativity" of the responses
stop_sequence = "\n" # Stop generating text after reaching a newline or maybe it could be none

In [ ]:
# function to get the ChatGPT response given a trigger
def get_chat_response(trigger):
    response = openai.Completion.create(
        engine = model_engine, # defined above
        prompt = trigger, # given prompt
        max_tokens = max_words,
        temperature = creativity,
        stop = stop_sequence,
    )
    chat_response = response.choices[0].text.strip()
    return chat_response.lower()

In [ ]:
# initial prompt to send to the openai
initial_prompt = """Two humans are playing a repeated prisoner’s dilemma game. 
                    In each round, each player needs to select between two options 
                    "keep" and "give". If both of them give, each will earn 1. 
                    If both keep, both of them earn nothing. If one gives the other 
                    one keeps, the one gives earns nothing, while the one keeps earns 
                    $1.5. While choosing the action, each player also needs to send a 
                    message of no more than 50 words to the other player. The message 
                    together with the action will be revealed to the other after the 
                    choice is made. After that, a new round will start. 
                    Now suppose you are one of the players, please choose an action and send a message. 
                    Response like this: Action: XXX Message: XXX. """

# with the initial prompt chatgpt will generate text to start to message.
chat_initial = get_chat_response(initial_prompt)
ai2_trigger = f"This is the other player's action and message: '{chat_initial}'"
ai2_initial_response = get_chat_response(ai2_trigger)



In [ ]:
# here we separate action and message

def action_message_finder (text):
    # Define regular expressions to match "action" and "message" followed by text
    action_regex = re.compile(r'action\s+(.*?)\s*message', re.IGNORECASE)
    message_regex = re.compile(r'message\s+(.*)', re.IGNORECASE)
    # Find the matches in the text generated
    action_match = action_regex.search(chat_initial)
    message_match = message_regex.search(chat_initial)

    return action_match, message_match





# First round data collection

In [ ]:
# saving the action and message of each round in a dataframe

chat = pandas.DataFrame()
chat[Action1], chat[Message1] = action_message_finder(chat_initial)
chat[Action2], chat[Message2] = action_message_finder(ai2_initial_response)


# we need to save the files along with round numer and the players name or number or code, in a json file.

# Round 2+

In [ ]:
# Here we will write the code for the other rounds

promt = f"""Two humans are playing a repeated prisoner’s dilemma game. In each round, each player will be given $1 and 
           needs to select between cooperate and defect. Choosing to cooperate will reduce a player’s own money by 
           $0.05 while increasing the other player’s payoff by $0.1. Choosing to defect will not change the payoff of the 
           player who has chosen this. 
           While choosing the action, each player also needs to send a message of no more than 50 words to the 
           opponent. The message together with the action will be revealed to the opponent after 
           the choice is made. After that, a new round will start.

           These are the actions and messages so far:
           Action: {}
           Message: {}

           Based on the the previous actions and messages continue the game and generate 
           the action and message for this round."""